# Print Runtime

In [1]:
sf=10

In [2]:
from psutil import *
vCPU = str(cpu_count()) + " vCPU"
mem=round(virtual_memory().total/(1024 * 1024 * 1024),0)
runtime = 'Lineitem-SF'+str(sf)+' ,Google Colab, '+vCPU+' '+str(mem)+'GB'
runtime

'Lineitem-SF10 ,Google Colab, 16 vCPU 59.0GB'

# Install Packages

In [3]:
!pip install pyarrow --upgrade
!pip install duckdb --pre --upgrade
!pip install polars
!pip install datafusion==0.8
!curl https://clickhouse.com/ | sh
#could not install pyspark, something to do with java or some stuff in that nature

  Using cached duckdb-0.7.2.dev1623-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (14.9 MB)
  Using cached polars-0.17.0-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.9 MB)
  Using cached datafusion-0.8.0-cp37-abi3-manylinux_2_31_x86_64.whl (12.7 MB)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2739    0  2739    0     0   8225      0 --:--:-- --:--:-- --:--:--  8225

Will download https://builds.clickhouse.com/master/amd64/clickhouse into clickhouse.2

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  530M  100  530M    0     0  34.7M      0  0:00:15  0:00:15 --:--:-- 39.1M

Successfully downloaded the ClickHouse binary, you can run it as:
    ./clickhouse.2

You can also install it:
    sudo ./clickhouse.2 install


# Generate Parquet File

In [4]:
%%time
import duckdb
import pathlib


for x in range(0, sf) :
  con=duckdb.connect()
  con.sql('PRAGMA disable_progress_bar;SET preserve_insertion_order=false')
  con.sql(f"CALL dbgen(sf={sf} , children ={sf}, step = {x})") 
  for tbl in ['lineitem'] :
     pathlib.Path(f'./{tbl}').mkdir(parents=True, exist_ok=True) 
     con.sql(f"COPY (SELECT * FROM {tbl}) TO './{tbl}/{x}.parquet' ")
  con.close()

CPU times: user 5min 4s, sys: 19.3 s, total: 5min 24s
Wall time: 2min 41s


# DataFusion

In [5]:
%%time
from datafusion import SessionContext
ctx = SessionContext()
ctx.register_parquet('lineitem', './lineitem/*.parquet')
df = ctx.sql("select * from lineitem order by l_shipdate")
df.write_parquet("lineitem_Datafusion")

CPU times: user 2min 16s, sys: 18 s, total: 2min 34s
Wall time: 1min 5s


# DuckDB

In [6]:
%%time
import duckdb 
duckdb.sql(
'''
PRAGMA disable_progress_bar;
COPY  (select * from './lineitem/*.parquet' order by l_shipdate )
to 'lineitem_DuckDB' (FORMAT 'PARQUET', PER_THREAD_OUTPUT TRUE)
''')

CPU times: user 3min 52s, sys: 39.3 s, total: 4min 31s
Wall time: 27.9 s


# Clickhouse

In [7]:
%%time
#Query 1
!./clickhouse local --query=" \
    select * FROM file ('./lineitem/*.parquet',Parquet) \
    order by l_shipdate \
    INTO OUTFILE 'clickhouse.parquet' \
    FORMAT Parquet \
    "

CPU times: user 1.91 s, sys: 614 ms, total: 2.53 s
Wall time: 1min 36s


# Polars

In [8]:
%%time
import polars as pl
pl.scan_parquet("./lineitem/*.parquet").sort("l_shipdate").sink_parquet("lineitem_sort_polars.parquet")

STREAMING CHUNK SIZE: 3125 rows


CPU times: user 2min 50s, sys: 40.5 s, total: 3min 31s
Wall time: 1min 15s


# Pyarrow

In [ ]:
%%time
import pyarrow.dataset as ds
lineitem = ds.dataset('./lineitem',format="parquet").sort_by('l_shipdate')
ds.write_dataset(lineitem,'./pyarrow', format="parquet")